# Convolutional Neural Network

Note the dataset is photos so is to big to run on google colab so needs to be run in jupiter notebook :(

### Importing the libraries

In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
#only applying transformations to training set to avoid overfitting.
#(image augmentation) This code can be seen on keras API

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range=0.2,
    horizontal_flip=True) #1./255 is essentially future scaling

#note training data is called training_set in dataset directory
#using (64,64) as its the final size of images that will be fed into network (bigger means slower)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
     target_size = (64, 64),
     batch_size = 32,
     class_mode = 'binary')


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
#need to apply the same future scaling but don't want to transform the images as they are new fresh images
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary') 

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential() 

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64,64,3]))
#kernal size is the size of the feature detectors
#again using rectifier function
#64,64,3 the 3 says its coloured as RGB

### Step 2 - Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#pool size is the size of the frame which is being pooled over
#strides is obvs the stride length taken by the pool area

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
#same as with ANN now
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) 
#binary output so only need one final neuron

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
#same as in last example with binary output
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
#again found epochs by trial and error (note with 25 still takes a while to run)

Epoch 1/25
250/250 [==============================] - 69s 273ms/step - loss: 0.6774 - accuracy: 0.5696 - val_loss: 0.6044 - val_accuracy: 0.6700
Epoch 2/25
250/250 [==============================] - 67s 268ms/step - loss: 0.6069 - accuracy: 0.6651 - val_loss: 0.5923 - val_accuracy: 0.6775
Epoch 3/25
250/250 [==============================] - 63s 253ms/step - loss: 0.5745 - accuracy: 0.7026 - val_loss: 0.5232 - val_accuracy: 0.7460
Epoch 4/25
250/250 [==============================] - 67s 266ms/step - loss: 0.5364 - accuracy: 0.7329 - val_loss: 0.5088 - val_accuracy: 0.7505
Epoch 5/25
250/250 [==============================] - 63s 253ms/step - loss: 0.5132 - accuracy: 0.7530 - val_loss: 0.5366 - val_accuracy: 0.7360
Epoch 6/25
250/250 [==============================] - 69s 275ms/step - loss: 0.4922 - accuracy: 0.7567 - val_loss: 0.4760 - val_accuracy: 0.7805
Epoch 7/25
250/250 [==============================] - 68s 270ms/step - loss: 0.4759 - accuracy: 0.7710 - val_loss: 0.4886 - val_ac

## Part 4 - Making a single prediction

In [15]:
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64,64))

#predict method expects a 2d array so need to convert this image into a 2d array
test_image = image.img_to_array(test_image)

#cnn was trained on batches of images so now we still have to input images as a batch
#So adding dimension so it fits what model expects (still only 1 element in bacth)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

#now figuring out what is 1/0 dog/cat
training_set.class_indices

if (result[0][0] == 1):
  prediction = 'dog'
else:
  prediction = 'cat'


In [16]:
print(prediction)

dog


Yay!! Correct prediction, I also tested on a few other images that all came back as correct,the 80% accuracy on the test set is preety good but not overfitted. If the images augmentation wasn't done then the accuracy would be really high on the training set but the model wouldn't work well on other data.